In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from bokeh.plotting import output_notebook, show
output_notebook()

Loading BokehJS ...

In [3]:
base = pd.read_csv("HR_comma_sep.csv")

In [7]:
train = base.sample(10000)
holdout_data = base[~base.index.isin(train.index)]
new_data = holdout_data.sample(2000).drop("left", axis=1)
holdout_data = holdout_data[~holdout_data.index.isin(new_data.index)]

In [8]:
train.shape
holdout_data.shape
new_data.shape

(10000, 10)

(2999, 10)

(2000, 9)

In [11]:
train.dtypes

satisfaction_level       float64
last_evaluation          float64
number_project             int64
average_montly_hours       int64
time_spend_company         int64
Work_accident              int64
left                       int64
promotion_last_5years      int64
sales                     object
salary                    object
dtype: object

In [17]:
import Model_selection, importlib

In [21]:
ms = Model_selection.model_train(modelBase=train, target="left", scoring="neg_log_loss", n_jobs=-1,
                                 categorical_columns=["Work_accident", "promotion_last_5years"])

In [22]:
ms.fit_models()

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 357 out of 500 | elapsed:   14.6s remaining:    5.8s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   20.1s finished


gbm best CV score: -0.0901562661924
gbm fitting complete

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 357 out of 500 | elapsed:    6.6s remaining:    2.6s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   11.1s finished


random_forest best CV score: -0.175546507169
random_forest fitting complete

Fitting 5 folds for each of 4 candidates, totalling 20 fits


...

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   48.9s finished


svm best CV score: -0.106678978317
svm fitting complete

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Done  12 out of  25 | elapsed:    0.3s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    0.5s finished


logistic_regression best CV score: -0.424707011161
logistic_regression fitting complete



In [23]:
ms.apply_models()
ms.evaluate_models()

gbm Training accuracy:98.98%
gbm Test accuracy:97.85%
gbm Training f1_score:97.81%
gbm Test f1_score:95.45%

random_forest Training accuracy:94.45%
random_forest Test accuracy:95.25%
random_forest Training f1_score:88.38%
random_forest Test f1_score:90.22%

svm Training accuracy:99.86%
svm Test accuracy:97.25%
svm Training f1_score:99.71%
svm Test f1_score:94.07%

logistic_regression Training accuracy:79.96%
logistic_regression Test accuracy:79.25%
logistic_regression Training f1_score:46.69%
logistic_regression Test f1_score:43.69%



In [27]:
ms.select_retrain_model(["gbm", "svm"])

In [28]:
ms.test_on_holdout_data(X=holdout_data.loc[:, holdout_data.columns!=ms.target], y = holdout_data.loc[:, ms.target])

Re-fitting on the full data...
Re-fitting on the full data...


In [29]:
ms.save_model_objects()

In [38]:
predictions = Model_selection.model_apply(new_data, model="gbm.pkl", predict_proba=True)

In [39]:
predictions

array([[  9.92061644e-01,   7.93835561e-03],
       [  9.97089142e-01,   2.91085757e-03],
       [  9.97543972e-01,   2.45602829e-03],
       ..., 
       [  5.14494029e-03,   9.94855060e-01],
       [  9.99004638e-01,   9.95361564e-04],
       [  8.92988605e-04,   9.99107011e-01]])